In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"auliamarshandaa","key":"314d2e9f60b2cbd8c5934a656e2e6e6b"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [3]:
!kaggle datasets download -d aadarshvelu/aids-virus-infection-prediction

Dataset URL: https://www.kaggle.com/datasets/aadarshvelu/aids-virus-infection-prediction
License(s): CC0-1.0
  0% 0.00/1.63M [00:00<?, ?B/s]
100% 1.63M/1.63M [00:00<00:00, 116MB/s]


In [4]:
!unzip aids-virus-infection-prediction.zip -d aids-virus-infection-prediction
!ls aids-virus-infection-prediction

Archive:  aids-virus-infection-prediction.zip
  inflating: aids-virus-infection-prediction/AIDS_Classification.csv  
  inflating: aids-virus-infection-prediction/AIDS_Classification_15000.csv  
  inflating: aids-virus-infection-prediction/AIDS_Classification_5000.csv  
  inflating: aids-virus-infection-prediction/AIDS_Classification_50000.csv  
AIDS_Classification_15000.csv  AIDS_Classification_5000.csv
AIDS_Classification_50000.csv  AIDS_Classification.csv


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score

import seaborn as sns
from keras.layers import Dense, BatchNormalization, Dropout, LSTM
from keras.models import Sequential
from keras.utils import to_categorical
from keras import callbacks

import tensorflow as tf
from tensorflow.keras.optimizers import Adam

In [6]:
df = pd.read_csv("/content/aids-virus-infection-prediction/AIDS_Classification.csv")

In [7]:
df.head()

,time,trt,age,wtkg,hemo,homo,drugs,karnof,oprior,z30,...,str2,strat,symptom,treat,offtrt,cd40,cd420,cd80,cd820,infected
0,948,2,48,89.8128,0,0,0,100,0,0,...,0,1,0,1,0,422,477,566,324,0
1,1002,3,61,49.4424,0,0,0,90,0,1,...,1,3,0,1,0,162,218,392,564,1
2,961,3,45,88.4520,0,1,1,90,0,1,...,1,3,0,1,1,326,274,2063,1893,0
3,1166,3,47,85.2768,0,1,0,100,0,1,...,1,3,0,1,0,287,394,1590,966,0
4,1090,0,43,66.6792,0,1,0,100,0,1,...,1,3,0,0,0,504,353,870,782,0


In [10]:
df.drop(columns=['trt','wtkg','drugs','karnof','oprior','z30','race','str2','strat','symptom','offtrt','cd40','cd420','cd80','cd820'], inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2139 entries, 0 to 2138
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   time      2139 non-null   int64
 1   age       2139 non-null   int64
 2   hemo      2139 non-null   int64
 3   homo      2139 non-null   int64
 4   preanti   2139 non-null   int64
 5   gender    2139 non-null   int64
 6   treat     2139 non-null   int64
 7   infected  2139 non-null   int64
dtypes: int64(8)
memory usage: 133.8 KB


In [12]:
import plotly.express as px
fig = px.histogram(df, x='infected')

fig.update_layout(title_text='<b>Count of Autistic and Non-Autistic Patients</b>', title_x=0.5)

fig.show()

In [13]:
X = df.drop("infected",axis=1)
Y = pd.get_dummies(df['infected'])

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25)

In [15]:
print(f"Shape of X_train is: {X_train.shape}")
print(f"Shape of Y_train is: {Y_train.shape}\n")
print(f"Shape of X_test is: {X_test.shape}")
print(f"Shape of Y_test is: {Y_test.shape}")

Shape of X_train is: (1604, 7)
Shape of Y_train is: (1604, 2)

Shape of X_test is: (535, 7)
Shape of Y_test is: (535, 2)


In [16]:
input_dim = X.shape[1]

In [17]:
model = Sequential()
model.add(Dense(8, input_dim = input_dim, kernel_initializer='normal', activation='relu'))
model.add(Dense(5, activation = "relu", kernel_initializer='normal'))
model.add(Dense(2, activation = 'sigmoid'))

In [18]:
model.compile(optimizer = Adam(learning_rate = 0.001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 64        
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 2)                 12        
                                                                 
Total params: 121 (484.00 Byte)
Trainable params: 121 (484.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
result = model.fit(X_train, Y_train, epochs = 15, batch_size = 10)

Epoch 1/15
161/161 [==============================] - 1s 2ms/step - loss: 0.5234 - accuracy: 0.7556
Epoch 2/15
161/161 [==============================] - 0s 2ms/step - loss: 0.4450 - accuracy: 0.7812
Epoch 3/15
161/161 [==============================] - 0s 2ms/step - loss: 0.4368 - accuracy: 0.7943
Epoch 4/15
161/161 [==============================] - 0s 3ms/step - loss: 0.4307 - accuracy: 0.7943
Epoch 5/15
161/161 [==============================] - 0s 2ms/step - loss: 0.4218 - accuracy: 0.7974
Epoch 6/15
161/161 [==============================] - 0s 2ms/step - loss: 0.4197 - accuracy: 0.8042
Epoch 7/15
161/161 [==============================] - 0s 2ms/step - loss: 0.4133 - accuracy: 0.8049
Epoch 8/15
161/161 [==============================] - 0s 3ms/step - loss: 0.4070 - accuracy: 0.8148
Epoch 9/15
161/161 [==============================] - 1s 4ms/step - loss: 0.4008 - accuracy: 0.8105
Epoch 10/15
161/161 [==============================] - 1s 3ms/step - loss: 0.3965 - accuracy: 0.8117

In [20]:
y_pred =model.predict(X_test)
y_pred = (y_pred>0.5)

17/17 [==============================] - 0s 2ms/step


In [21]:
accuracy = accuracy_score(Y_test,y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.7981308411214953


In [22]:
model.save('./AidsVirus.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



In [23]:
model = tf.keras.models.load_model('./AidsVirus.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open("./AidsVirus.tflite", "wb").write(tflite_model)

2548